# Review: Improved Training of Wasserstein GANs

###### ***Author***

### 1. Contribution

- critic weight clipping의 문제점을 toy dataset을 통해서 보였다.
- gradient penalty를 제안해서 위의 문제를 해결했다.
- 다양한 GAN 구조에 대해 안정적인 학습을 증명하고, weight clipping에 대한 성능 향상, high quality image generation 등이 가능하다.

### 2. Background & Difficulties with weight constraints

#### A. Background
<figure>
![Untitled](https://user-images.githubusercontent.com/68529301/165379661-bd34fa9b-6e08-4f7a-992c-0808f69839b6.png)
<figcaption align = "center"><b>Fig.1 - 4K Mountains Wallpaper</b></figcaption>
</figure>

<figure>
![Eq 1](https://user-images.githubusercontent.com/68529301/165379678-79569b92-84a8-4fe5-93ad-993d6fbbf4b8.png)
<figcaption align = "center"><b>Fig.1 - 4K Mountains Wallpaper</b></figcaption>
</figure>

<figure>
![Eq 2](https://user-images.githubusercontent.com/68529301/165379682-b07f66d7-5a84-4a28-b526-320ef27d4608.png)
<figcaption align = "center"><b>Fig.1 - 4K Mountains Wallpaper</b></figcaption>
</figure>

<figure>
![Eq3](https://user-images.githubusercontent.com/68529301/165379684-63010cfa-e135-44bb-a374-8c321f178f39.png)
<figcaption align = "center"><b>Fig.1 - 4K Mountains Wallpaper</b></figcaption>
</figure>

WGAN 손실함수(Eq1)을 max로 만드는 f를 f*라고 하자. 이 f*는 Eq2를 통해서 구할 수 있다. 

$x$ ~$P_g$, $y$~$P_r$로 샘플링을 해서 $x$와 $y$를 보간한 직선 중 $x$와 $y$ 사이의 점 $x_t$가 있다고 할 때, $x_t=tx+(1-t)y$ 로 나타낼 수 있다. (0 ≤ $t$ ≤ 1)

이때, 어떠한 $x_t$이든 Eq3이 만족된다. (증명은 appendix)

![Untitled](https://user-images.githubusercontent.com/68529301/165379686-ea77e61a-74a4-485e-bed3-7c09c058fb1b.png)

## B. Weight Clipping의 문제점

- weight clipping: 업데이트된 weight를 특정 구간으로 제한하는 것 → 1-Lipschitz를 만족시키기 위함
- weight clippping이 최적화에 문제를 발생시킨다. optimization이 성공했을때조차 critic이 pathological value surface를 가지는 경우도 있다.

![윗줄(weight clipping): 데이터 분포의 higher moments를 잡아내는 데 실패했다. // 밑줄(gradient panelty)](https://user-images.githubusercontent.com/68529301/165379691-dbfc8dbe-2445-4022-9853-de05f036ef98.png)

윗줄(weight clipping): 데이터 분포의 higher moments를 잡아내는 데 실패했다. // 밑줄(gradient panelty)

![Swiss Roll dataset을 학습시키는 도중 weight clipping을 사용할 때, 발생하는 vanish / explode를 보여주고 gradient penalty를 사용 시 문제가 발생하지 않는 것을 보여주는 gradient norm](https://user-images.githubusercontent.com/68529301/165379692-6eec3444-0851-4cc2-bbb4-82f808c8d4dc.png)

Swiss Roll dataset을 학습시키는 도중 weight clipping을 사용할 때, 발생하는 vanish / explode를 보여주고 gradient penalty를 사용 시 문제가 발생하지 않는 것을 보여주는 gradient norm

![weight clipping은 가중치가 clipping boundary 근처로 몰리지만 gradient panelty는 그렇지 않는 것을 알 수 있다. ](https://user-images.githubusercontent.com/68529301/165379693-389b4474-4854-4bc0-bf5c-8ca20016e014.png)

weight clipping은 가중치가 clipping boundary 근처로 몰리지만 gradient panelty는 그렇지 않는 것을 알 수 있다.